In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def read_data(file_paths):
    """
    Read data from multiple files and concatenate durations.
    
    Args:
        file_paths: List of file paths to read data from
        
    Returns:
        durations: Concatenated array of duration values
    """
    all_durations = []
    
    for file_path in file_paths:
        data = pd.read_csv(file_path, sep='\t', header=None, names=['timestamps', 'durations'])
        all_durations.append(data['durations'].values)
    
    return np.concatenate(all_durations)

In [ ]:
file_paths = []
data = read_data(file_paths)
mean = np.mean(file_paths)

plt.plot(data)
plt.axvline(mean, label =f'Mean = {mean:.2f}')
